In [ ]:
# Mount Google Drive if using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [8]:

# Define the paths to your image folders
motion_blur_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/motion_blurred"
defocused_blur_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/defocused_blurred"
sharp_images_path = "/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/sharp"


In [9]:


# Define batch size and image size
batch_size = 32
img_size = (224, 224)  # You can adjust this based on your model's input size

# Create an ImageDataGenerator for loading and augmenting images
datagen = ImageDataGenerator(
    rescale=1./255,         # Normalize pixel values to [0, 1]
    validation_split=0.2,   # Reserve 20% of data for validation
    horizontal_flip=True,   # Augmentation: flip images horizontally
    rotation_range=20,      # Augmentation: rotate images
    zoom_range=0.2          # Augmentation: zoom in and out
)

# Load the images from the directory, creating training and validation sets
train_data = datagen.flow_from_directory(
    directory="/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # One-hot encoding for multi-class classification
    subset='training'          # Training data
)

val_data = datagen.flow_from_directory(
    directory="/content/drive/MyDrive/Google Colab/Machine_Learning/data/thesis/deep_learning/",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # One-hot encoding for multi-class classification
    subset='validation'        # Validation data
)

# Print the class indices to ensure correct labeling
print("Class indices:", train_data.class_indices)

# Automatically detect the number of classes
num_classes = len(train_data.class_indices)

# Build the CNN model
model = models.Sequential()

# Add convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers before passing it to fully connected layers
model.add(layers.Flatten())

# Fully connected (dense) layers
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))  # Add dropout for regularization to prevent overfitting

# Output layer: use softmax activation for multi-class classification
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,   # Adjust the number of epochs based on your needs
    steps_per_epoch=train_data.samples // batch_size,
    validation_steps=val_data.samples // batch_size
)

# Save the trained model to a file
model.save('blur_classification_model.h5')

# Evaluate the model on the validation data
loss, accuracy = model.evaluate(val_data)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


Found 840 images belonging to 3 classes.
Found 210 images belonging to 3 classes.
Class indices: {'defocused_blurred': 0, 'motion_blurred': 1, 'sharp': 2}


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │       9,437,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,680,067 (36.93 MB)

 Trainable params: 9,680,067 (36.93 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 273s 9s/step - accuracy: 0.3184 - loss: 1.1223 - val_accuracy: 0.4271 - val_loss: 1.0832
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.4062 - loss: 1.0904 - val_accuracy: 0.6111 - val_loss: 1.0441
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 197s 5s/step - accuracy: 0.4366 - loss: 1.0454 - val_accuracy: 0.4167 - val_loss: 0.9881
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 851ms/step - accuracy: 0.5312 - loss: 0.9194 - val_accuracy: 0.5556 - val_loss: 1.0217
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 182s 6s/step - accuracy: 0.4993 - loss: 0.9267 - val_accuracy: 0.5052 - val_loss: 1.0443
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.5625 - loss: 1.1744 - val_accuracy: 0.6667 - val_loss: 0.6034
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 195s 5s/step - accuracy: 0.5634 - loss: 0.8403 - val_accuracy: 0.6198 - val_loss: 0.8631
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.6250 - loss: 0.6980 - val_accuracy: 0.6111 -

7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.6638 - loss: 0.9393
Validation Accuracy: 62.86%
